# Conv2D as a pseudo-Conv1D

    This notebook is just as for a playground. Cihan says that he tried to explain it with videos, we need to understand it by doing matrix operations in a paper to understand the concept and what is going on.

In [65]:
import numpy as np
 # ML Algorithms to be used
import tensorflow as tf
from tensorflow import keras
from keras import optimizers, models, layers, regularizers
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D

Imagine that we have a time series data, with a shape of 10. It could be the last 10 sensor measurement. Below, we will create an array for tensorflow:

[batch size, image size, # channels / features]

This time, we will have a batch size of 1, time series of shape 10x1, and only one channel.

Let's break down the shape `(1, 10, 1, 1)`:

- `1`: The first dimension implies that there is one example or batch in your dataset. This can also be thought of as the number of instances or records you have.

- `10`: The second dimension implies that there are 10 different features, or time steps, or rows in each example or instance.

- `1`: The third dimension implies that there is one column in each feature or time step.

- `1`: The fourth dimension, usually reserved for channels in image processing (e.g., RGB channels for color images), denotes here that there is one channel or feature for each instance.


In [2]:
#Creating a dummy numpy array for testing kernels
X_train = np.array([[ [[_]] for _ in range(10)]])
X_train.shape

(1, 10, 1, 1)

In [3]:
print(X_train)

[[[[0]]

  [[1]]

  [[2]]

  [[3]]

  [[4]]

  [[5]]

  [[6]]

  [[7]]

  [[8]]

  [[9]]]]


In [4]:
def build_model_CNN():
  # input array shape for the network:
  input_shape = X_train.shape[1],X_train.shape[2],X_train.shape[3]
  #Here we will use Sequential API like we did in MLP
  model = models.Sequential([
  # Now we will introduce the layers. We will start with the convolution layer:
  Conv2D(1,(2,1),(1,1),padding='valid',activation="linear",input_shape=input_shape),
  #Adding the pooling layer:
  #MaxPooling2D((2,2),padding='same'),
  ])
  model.compile(optimizer='Adam',loss = "binary_crossentropy", metrics=["accuracy"]) 
  return model

`Conv2D(1,(2,1),(1,1),padding='valid',activation="linear",input_shape=input_shape),`

This is a 2D convolution layer, and the parameters are as follows:

- `1`: The first argument is the number of filters that the convolution layer will learn. Filters are the "feature detectors", for example, edge detectors, blob detectors, etc. Each filter represents a specific feature it's learned to detect.

- `(2,1)`: This tuple defines the kernel size, that is, the width and height of the convolution window. In this case, the convolution window is of size 2x1.

- `(1,1)`: This tuple is the strides of the convolution along the width and height. Strides mean the steps by which the convolution window moves across the input. Here, the convolution window moves 1 step at a time horizontally and vertically.

- `padding='valid'`: This means there is no padding around the input or feature map, implying that the dimensions may change after the convolution operation. In contrast, 'same' padding implies that padding is used so the output feature maps have the same width and height as the input.

- `activation='linear'`: This is the activation function for the layer. An activation function is used to introduce non-linearity into the network. The 'linear' activation function is essentially no activation function (f(x) = x), meaning the output equals the input.

- `input_shape=input_shape`: This is the shape of the input data that the layer will receive. The `input_shape` parameter is typically only required for the first layer of the model. In your case, the input shape is the same as the shape of your training data.


In [5]:
cnn = build_model_CNN()
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 9, 1, 1)           3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [6]:
weights = cnn.get_weights()
print(weights[0])
print(weights[1])

[[[[0.3996029]]]


 [[[0.9845456]]]]
[0.]


## Define kernel

In [7]:
filter_1 = -0.1
filter_2 = 1

In [8]:
custom_kernel_1 = np.array([[[[filter_1]]],[[[filter_2]]]])
custom_kernel_1

array([[[[-0.1]]],


       [[[ 1. ]]]])

In [9]:
bias_1 = np.array([0])
bias_1

array([0])

# Filling up kernel for debugging purposes

In [10]:
# Creating custom weight cnn:
custom_cnn_1 = build_model_CNN()
custom_cnn_1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 9, 1, 1)           3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [11]:
custom_cnn_1.set_weights([custom_kernel_1,bias_1])
weights = custom_cnn_1.get_weights()
print(weights[0])
print(weights[1])

[[[[-0.1]]]


 [[[ 1. ]]]]
[0.]


# Conv. operation

In [12]:
X_train

array([[[[0]],

        [[1]],

        [[2]],

        [[3]],

        [[4]],

        [[5]],

        [[6]],

        [[7]],

        [[8]],

        [[9]]]])

In [13]:
y_calculated_1 = custom_cnn_1.predict(X_train)
y_calculated_1

1/1 [==============================] - 11s 11s/step


array([[[[1. ]],

        [[1.9]],

        [[2.8]],

        [[3.7]],

        [[4.6]],

        [[5.5]],

        [[6.4]],

        [[7.3]],

        [[8.2]]]], dtype=float32)

# Flipping kernel dimensions

Flipped kernel do not work and raise an error, as the dimensions do not match. But if you use padding, it seems to use that dimension as input:

In [14]:
def build_model_CNN_2():
  # input array shape for the network:
  input_shape = X_train.shape[1],X_train.shape[2],X_train.shape[3]
  #Here we will use Sequential API like we did in MLP
  model = models.Sequential([
  # Now we will introduce the layers. We will start with the convolution layer:
  Conv2D(1,(1,2),(1,1),padding='valid',activation="linear",input_shape=input_shape),
  #Adding the pooling layer:
  #MaxPooling2D((2,2),padding='same'),
  ])  
  model.compile(optimizer='Adam',loss = "binary_crossentropy", metrics=["accuracy"]) 
  return model

In [15]:
cnn_2 = build_model_CNN_2()
cnn_2.summary()

ValueError: Exception encountered when calling layer "conv2d_2" (type Conv2D).

Negative dimension size caused by subtracting 2 from 1 for '{{node conv2d_2/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](Placeholder, conv2d_2/Conv2D/ReadVariableOp)' with input shapes: [?,10,1,1], [1,2,1,1].

Call arguments received by layer "conv2d_2" (type Conv2D):
  • inputs=tf.Tensor(shape=(None, 10, 1, 1), dtype=float32)

Adding padding just to show:

In [16]:
def build_model_CNN_2():
  # input array shape for the network:
  input_shape = X_train.shape[1],X_train.shape[2],X_train.shape[3]
  #Here we will use Sequential API like we did in MLP
  model = models.Sequential([
  # Now we will introduce the layers. We will start with the convolution layer:
  Conv2D(1,(1,2),(1,1),padding='same',activation="linear",input_shape=input_shape),
  #Adding the pooling layer:
  #MaxPooling2D((2,2),padding='same'),
  ])  
  model.compile(optimizer='Adam',loss = "binary_crossentropy", metrics=["accuracy"]) 
  return model

In [17]:
cnn_2 = build_model_CNN_2()
cnn_2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 10, 1, 1)          3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [18]:
weights = cnn_2.get_weights()
print(weights[0])
print(weights[1])

[[[[0.77284503]]

  [[0.41656208]]]]
[0.]


In [19]:
custom_kernel_2 = np.array([[[[filter_1]],[[filter_2]]]])
custom_kernel_2

array([[[[-0.1]],

        [[ 1. ]]]])

In [20]:
bias_2 = np.array([0])

In [21]:
# Creating custom weight cnn:
custom_cnn_2 = build_model_CNN_2()
custom_cnn_2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 10, 1, 1)          3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [22]:
custom_cnn_2.set_weights([custom_kernel_2,bias_2])
weights = custom_cnn_2.get_weights()
print(weights[0])
print(weights[1])

[[[[-0.1]]

  [[ 1. ]]]]
[0.]


So, if you are using 2D conv. layers for 1D data, make sure that it is prepared correctly.

# Fixing it: proper 1D kernels

In [23]:
def build_model_CNN_2():
  # input array shape for the network:
  input_shape = X_train.shape[1],X_train.shape[2],X_train.shape[3]
  #Here we will use Sequential API like we did in MLP
  model = models.Sequential([
  # Now we will introduce the layers. We will start with the convolution layer:
  Conv2D(1,(2,1),(1,1),padding='valid',activation="linear",input_shape=input_shape),
  #Adding the pooling layer:
  #MaxPooling2D((2,2),padding='same'),
  ])  
  model.compile(optimizer='Adam',loss = "binary_crossentropy", metrics=["accuracy"]) 
  return model

In [24]:
cnn_2 = build_model_CNN_2()
cnn_2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 9, 1, 1)           3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [25]:
weights = cnn_2.get_weights()
print(weights[0])
print(weights[1])

[[[[-0.9016404 ]]]


 [[[-0.82011986]]]]
[0.]


In [26]:
custom_kernel_2 = np.array([[[[filter_1]]],[[[filter_2]]]])
custom_kernel_2

array([[[[-0.1]]],


       [[[ 1. ]]]])

In [27]:
bias_2 = np.array([0.3])

In [28]:
# Creating custom weight cnn:
custom_cnn_2 = build_model_CNN_2()
custom_cnn_2.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 9, 1, 1)           3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [29]:
custom_cnn_2.set_weights([custom_kernel_2,bias_2])
weights = custom_cnn_2.get_weights()
print(weights[0])
print(weights[1])

[[[[-0.1]]]


 [[[ 1. ]]]]
[0.3]


# Conv. operation

In [30]:
X_train

array([[[[0]],

        [[1]],

        [[2]],

        [[3]],

        [[4]],

        [[5]],

        [[6]],

        [[7]],

        [[8]],

        [[9]]]])

In [31]:
y_calculated_2 = custom_cnn_2.predict(X_train)
y_calculated_2

1/1 [==============================] - 0s 77ms/step


array([[[[1.3      ]],

        [[2.2      ]],

        [[3.1      ]],

        [[4.       ]],

        [[4.9      ]],

        [[5.8      ]],

        [[6.7000003]],

        [[7.6000004]],

        [[8.5      ]]]], dtype=float32)

# 3 Filters with no padding

In [32]:
def build_model_CNN_3():
  # input array shape for the network:
  input_shape = X_train.shape[1],X_train.shape[2],X_train.shape[3]
  #Here we will use Sequential API like we did in MLP
  model = models.Sequential([
  # Now we will introduce the layers. We will start with the convolution layer:
  Conv2D(3,(2,1),(1,1),padding='valid',activation="linear",input_shape=input_shape),
  #Adding the pooling layer:
  #MaxPooling2D((2,2),padding='same'),
  ])  
  model.compile(optimizer='Adam',loss = "binary_crossentropy", metrics=["accuracy"]) 
  return model

In [33]:
custom_cnn_3 = build_model_CNN_3()
custom_cnn_3.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 9, 1, 3)           9         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [34]:
weights = custom_cnn_3.get_weights()
print(weights[0])
print(weights[1])

[[[[ 0.2561075   0.8587766  -0.24401975]]]


 [[[-0.05750662  0.84049886  0.8058812 ]]]]
[0. 0. 0.]


In [35]:
#kernel shape: how kernel data is saved. we will use our own for debugging.
weights[0].shape

(2, 1, 1, 3)

In [36]:
custom_kernel_3 = np.array([[[[ -0.1, 0.1 , 0.5 ]]],\
                            [[[  1  , 1   , 0.5 ]]]])
bias_3 = np.array([0,0,0])

In [37]:
custom_cnn_3.set_weights([custom_kernel_3,bias_3])
weights = custom_cnn_3.get_weights()
print(weights[0])
print(weights[1])

[[[[-0.1  0.1  0.5]]]


 [[[ 1.   1.   0.5]]]]
[0. 0. 0.]


In [38]:
y_3kernel_no_p = custom_cnn_3.predict(X_train)
y_3kernel_no_p

1/1 [==============================] - 0s 105ms/step


array([[[[1. , 1. , 0.5]],

        [[1.9, 2.1, 1.5]],

        [[2.8, 3.2, 2.5]],

        [[3.7, 4.3, 3.5]],

        [[4.6, 5.4, 4.5]],

        [[5.5, 6.5, 5.5]],

        [[6.4, 7.6, 6.5]],

        [[7.3, 8.7, 7.5]],

        [[8.2, 9.8, 8.5]]]], dtype=float32)

# 3 Filters with pooling

In [39]:
def build_model_CNN_3():
  # input array shape for the network:
  input_shape = X_train.shape[1],X_train.shape[2],X_train.shape[3]
  #Here we will use Sequential API like we did in MLP
  model = models.Sequential([
  # Now we will introduce the layers. We will start with the convolution layer:
  Conv2D(3,(2,1),(1,1),padding='valid',activation="linear",input_shape=input_shape),
  #Adding the pooling layer:
  MaxPooling2D((2,1),padding='valid'),
  ])  
  model.compile(optimizer='Adam',loss = "binary_crossentropy", metrics=["accuracy"]) 
  return model

In [40]:
custom_cnn_3 = build_model_CNN_3()
custom_cnn_3.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 9, 1, 3)           9         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 4, 1, 3)          0         
 )                                                               
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [41]:
weights = custom_cnn_3.get_weights()
print(weights[0])
print(weights[1])

[[[[-0.3257659   0.13477999 -0.48170936]]]


 [[[-0.08136684 -0.02073312 -0.83403826]]]]
[0. 0. 0.]


In [42]:
#kernel shape: how kernel data is saved. we will use our own for debugging.
weights[0].shape

(2, 1, 1, 3)

In [43]:
custom_kernel_3 = np.array([[[[ -0.1, 0.1 , 0.5 ]]],\
                            [[[  1  , 1   , 0.5 ]]]])
bias_3 = np.array([0,0,0])

In [44]:
custom_cnn_3.set_weights([custom_kernel_3,bias_3])
weights = custom_cnn_3.get_weights()
print(weights[0])
print(weights[1])

[[[[-0.1  0.1  0.5]]]


 [[[ 1.   1.   0.5]]]]
[0. 0. 0.]


In [45]:
y_3kernel_w_p = custom_cnn_3.predict(X_train)
y_3kernel_w_p

1/1 [==============================] - 0s 84ms/step


array([[[[1.9, 2.1, 1.5]],

        [[3.7, 4.3, 3.5]],

        [[5.5, 6.5, 5.5]],

        [[7.3, 8.7, 7.5]]]], dtype=float32)

In [46]:
y_3kernel_no_p

array([[[[1. , 1. , 0.5]],

        [[1.9, 2.1, 1.5]],

        [[2.8, 3.2, 2.5]],

        [[3.7, 4.3, 3.5]],

        [[4.6, 5.4, 4.5]],

        [[5.5, 6.5, 5.5]],

        [[6.4, 7.6, 6.5]],

        [[7.3, 8.7, 7.5]],

        [[8.2, 9.8, 8.5]]]], dtype=float32)

# 3 Filters with average pooling

In [47]:
def build_model_CNN_3():
  # input array shape for the network:
  input_shape = X_train.shape[1],X_train.shape[2],X_train.shape[3]
  #Here we will use Sequential API like we did in MLP
  model = models.Sequential([
  # Now we will introduce the layers. We will start with the convolution layer:
  Conv2D(3,(2,1),(1,1),padding='valid',activation="linear",input_shape=input_shape),
  #Adding the pooling layer:
  AveragePooling2D((2,1),padding='valid'),
  ])  
  model.compile(optimizer='Adam',loss = "binary_crossentropy", metrics=["accuracy"]) 
  return model

In [48]:
custom_cnn_3 = build_model_CNN_3()
custom_cnn_3.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 9, 1, 3)           9         
                                                                 
 average_pooling2d (AverageP  (None, 4, 1, 3)          0         
 ooling2D)                                                       
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [49]:
weights = custom_cnn_3.get_weights()
print(weights[0])
print(weights[1])

[[[[-0.423885   -0.2591138   0.0437749 ]]]


 [[[-0.5341716  -0.04940325 -0.37193984]]]]
[0. 0. 0.]


In [50]:
#kernel shape: how kernel data is saved. we will use our own for debugging.
weights[0].shape

(2, 1, 1, 3)

In [51]:
custom_kernel_3 = np.array([[[[ -0.1, 0.1 , 0.5 ]]],\
                            [[[  1  , 1   , 0.5 ]]]])
bias_3 = np.array([0,0,0])

In [52]:
custom_cnn_3.set_weights([custom_kernel_3,bias_3])
weights = custom_cnn_3.get_weights()
print(weights[0])
print(weights[1])

[[[[-0.1  0.1  0.5]]]


 [[[ 1.   1.   0.5]]]]
[0. 0. 0.]


In [53]:
y_3kernel_w_ap = custom_cnn_3.predict(X_train)
y_3kernel_w_ap

1/1 [==============================] - 0s 82ms/step


array([[[[1.45     , 1.55     , 1.       ]],

        [[3.25     , 3.75     , 3.       ]],

        [[5.05     , 5.95     , 5.       ]],

        [[6.8500004, 8.15     , 7.       ]]]], dtype=float32)

In [54]:
y_3kernel_no_p

array([[[[1. , 1. , 0.5]],

        [[1.9, 2.1, 1.5]],

        [[2.8, 3.2, 2.5]],

        [[3.7, 4.3, 3.5]],

        [[4.6, 5.4, 4.5]],

        [[5.5, 6.5, 5.5]],

        [[6.4, 7.6, 6.5]],

        [[7.3, 8.7, 7.5]],

        [[8.2, 9.8, 8.5]]]], dtype=float32)

# 2 Input channels

In [55]:
#Creating a dummy numpy array for testing kernels
X_train = np.array([[ [[_, 9-_]] for _ in range(10)]])
X_train.shape

(1, 10, 1, 2)

In [56]:
print(X_train)

[[[[0 9]]

  [[1 8]]

  [[2 7]]

  [[3 6]]

  [[4 5]]

  [[5 4]]

  [[6 3]]

  [[7 2]]

  [[8 1]]

  [[9 0]]]]


In [57]:
def build_model_CNN_2():
  # input array shape for the network:
  input_shape = X_train.shape[1],X_train.shape[2],X_train.shape[3]
  #Here we will use Sequential API like we did in MLP
  model = models.Sequential([
  # Now we will introduce the layers. We will start with the convolution layer:
  Conv2D(1,(2,1),(1,1),padding='valid',activation="linear",input_shape=input_shape),
  #Adding the pooling layer:
  #MaxPooling2D((2,2),padding='same'),
  ])  
  model.compile(optimizer='Adam',loss = "binary_crossentropy", metrics=["accuracy"]) 
  return model

In [58]:
cnn_2 = build_model_CNN_2()
cnn_2.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 9, 1, 1)           5         
                                                                 
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


In [59]:
weights = cnn_2.get_weights()
print(weights[0])
print(weights[1])

[[[[-0.5516615 ]
   [-0.5993333 ]]]


 [[[-0.9321215 ]
   [-0.15790772]]]]
[0.]


In [60]:
custom_kernel_2 = np.array([[[[0.1],[ -0.1 ]]],[[[1],[ -1]]]])
custom_kernel_2

array([[[[ 0.1],
         [-0.1]]],


       [[[ 1. ],
         [-1. ]]]])

In [61]:
bias_2 = np.array([0.3])

In [62]:
# Creating custom weight cnn:
custom_cnn_2 = build_model_CNN_2()
custom_cnn_2.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 9, 1, 1)           5         
                                                                 
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


In [63]:
custom_cnn_2.set_weights([custom_kernel_2,bias_2])
weights = custom_cnn_2.get_weights()
print(weights[0])
print(weights[1])

[[[[ 0.1]
   [-0.1]]]


 [[[ 1. ]
   [-1. ]]]]
[0.3]


## Conv. operation

In [64]:
y_calculated_2 = custom_cnn_2.predict(X_train)
y_calculated_2

1/1 [==============================] - 0s 111ms/step


array([[[[-7.6       ]],

        [[-5.3999996 ]],

        [[-3.2       ]],

        [[-0.99999994]],

        [[ 1.2       ]],

        [[ 3.3999999 ]],

        [[ 5.6000004 ]],

        [[ 7.8       ]],

        [[10.        ]]]], dtype=float32)

    Then, let's create a random input image of shape (1, 10, 10, 3), where 1 denotes the batch size (number of images), 10x10 is the image size (height x width), and 3 stands for the number of channels (RGB):

In [66]:
input_image = np.random.rand(1, 10, 10, 3)

    Now, let's define a model. First, we will use a convolution layer with kernel size of (3, 3), stride of (1,1), valid padding and linear activation function:

In [67]:
model = Sequential()
model.add(Conv2D(1, (3,3), strides=(1,1), padding='valid', activation='linear', input_shape=(10,10,3)))

    Conv2D(1, (3,3), strides=(1,1), padding='valid', activation='linear', input_shape=(10,10,3)) creates a convolution layer. The arguments are:

    1: Number of filters in the convolution.

    (3, 3): Size of the kernel that is used for the convolution.

    strides=(1, 1): Stride of the convolution along the height and width.

    padding='valid': No padding is applied.

    activation='linear': Linear activation function is used.

    To add padding, we can use the ZeroPadding2D layer. Let's add a zero-padding layer that adds 1 pixel of padding to the height and width of the image:

In [68]:
model.add(ZeroPadding2D(padding=(1,1)))

    Now, let's add a max pooling layer with pool size of (2, 2):

In [69]:
model.add(MaxPooling2D(pool_size=(2, 2)))

    MaxPooling2D(pool_size=(2, 2)) creates a max pooling layer. Here, (2, 2) is the size of the max pooling window.

    Similarly, let's add an average pooling layer:

In [ ]:
model.add(AveragePooling2D(pool_size=(2, 2)))

    To see the model summary:

In [70]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 8, 8, 1)           28        
                                                                 
 zero_padding2d (ZeroPadding  (None, 10, 10, 1)        0         
 2D)                                                             
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 1)          0         
 2D)                                                             
                                                                 
Total params: 28
Trainable params: 28
Non-trainable params: 0
_________________________________________________________________


In [71]:
output_image = model.predict(input_image)

1/1 [==============================] - 0s 140ms/step


    This is a very basic example that demonstrates the use of different types of layers in a CNN. By changing the parameters of the layers or by commenting out some of the layers, you can observe the changes in the output shape and better understand what each layer does.

    To understand kernels, consider this: in the convolution operation, kernels are like small windows that slide over the input image and perform element-wise multiplication and summation to generate a new feature map. Kernels are essentially the feature detectors of the network, and they learn to detect different features (e.g., edges, corners, textures) from the input.

    Padding is the process of adding extra pixels around the input image. This can be done to preserve the spatial dimensions of the input through the convolution operation, especially when using larger kernel sizes.